<a href="https://colab.research.google.com/github/wel51x/DS-Unit-4-Sprint-2-NLP/blob/master/DS42SC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2 Sprint Challenge*

# Natural Language Processing

**Part 1 - Working with Text Data**
Use Python string methods remove irregular whitespace from the following string:

In [0]:
#imports
import pandas as pd
import numpy as np
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report, roc_auc_score
from sklearn.naive_bayes import MultinomialNB

from gensim.models.word2vec import Word2Vec

In [2]:
# Downloads
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
whitespace_string = "\n\n  This is a    string   that has  \n a lot of  extra \n   whitespace.   "

print(whitespace_string)



  This is a    string   that has  
 a lot of  extra 
   whitespace.   


In [4]:
my_str = re.sub('[\s]+', ' ', whitespace_string.strip())
my_str.replace('has a lot of', 'doesn\'t have')

"This is a string that doesn't have extra whitespace."

### Use Regular Expressions to take the dates in the following .txt file and put them into a dataframe with columns for:

[RegEx dates.txt](https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt)

- Day
- Month
- Year


In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt',
                names = ["Month", "Day", "Year"], header = None, sep = "(?:,|\s)\s*")
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,Month,Day,Year
0,March,8,2015
1,March,15,2015
2,March,22,2015
3,March,29,2015
4,April,5,2015
5,April,12,2015
6,April,19,2015
7,April,26,2015
8,May,3,2015
9,May,10,2015


# Part 2 - Bag of Words 

### Use the twitter sentiment analysis dataset found at this link for the remainder of the Sprint Challenge:

[Twitter Sentiment Analysis Dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv)

 ### Clean and tokenize the documents ensuring the following properties of the text:

1) Text should be lowercase.

2) Stopwords should be removed.

3) Punctuation should be removed.

4) Tweets should be tokenized at the word level. 

(The above don't necessarily need to be completed in that specific order.)

### Output some cleaned tweets so that we can see that you made all of the above changes.


In [10]:
twitter = pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv")
print(twitter.shape)
twitter.head(11)

(99989, 2)


,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...
5,0,or i just worry too much?
6,1,Juuuuuuuuuuuuuuuuussssst Chillin!!
7,0,Sunny Again Work Tomorrow :-| ...
8,1,handed in my uniform today . i miss you ...
9,1,hmmmm.... i wonder how she my number @-)


In [11]:
# Stop words
stop = stopwords.words('english')

def clean_twitter(in_text):
  return ''.join(re.sub("(([^0-9A-Za-z \t])|@[A-Za-z0-9]+)|(\w+:\ / \ / \S+)","", in_text.lower()))

# Clean text
twitter['SentimentText'] = twitter['SentimentText'].apply(clean_twitter)
twitter = twitter.rename(columns={"SentimentText": "CleanedText"})

# Remove stop words
twitter['CleanedText'] = twitter['CleanedText'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# Tokenize
twitter['Tokens'] = twitter['CleanedText'].apply(word_tokenize)

twitter.head(11)

,Sentiment,CleanedText,Tokens
0,0,sad apl friend,"[sad, apl, friend]"
1,0,missed new moon trailer,"[missed, new, moon, trailer]"
2,1,omg already 730,"[omg, already, 730]"
3,0,omgaga im sooo im gunna cry ive dentist since ...,"[omgaga, im, sooo, im, gunna, cry, ive, dentis..."
4,0,think mi bf cheating tt,"[think, mi, bf, cheating, tt]"
5,0,worry much,"[worry, much]"
6,1,juuuuuuuuuuuuuuuuussssst chillin,"[juuuuuuuuuuuuuuuuussssst, chillin]"
7,0,sunny work tomorrow tv tonight,"[sunny, work, tomorrow, tv, tonight]"
8,1,handed uniform today miss already,"[handed, uniform, today, miss, already]"
9,1,hmmmm wonder number,"[hmmmm, wonder, number]"


### How should TF-IDF scores be interpreted? How are they calculated?

TF-IDF (Term Frequency - Inverse Document Frequency) scores help in determining the importance of a word in a document. The idea is that the more often a word occurs, the more important it is.

However, sometimes a word so frequently in a corpus of documents that its importance is diminished to the point where it becomes trivial, and should be removed via stop words.

For example, if the word "criminal" occurs several times in a document, it's likely to be important. However if I'm looking through the district attorney's files, that word is going to be less important as it will likely appear in most - if not all - documents.

Term Frequency (the TF in TF-IDF) is the number of times a word occurs in a given document.

Inverse Document Frequency (the IDF in TF-IDF) is calculated as the log of the total number of documents containing the expression divided by the number of documents in the corpus. containing the expression

# Part 3 - Document Classification

1) Use Train_Test_Split to create train and test datasets.

2) Vectorize the tokenized documents using your choice of vectorization method. 

 - Stretch goal: Use both of the methods that we talked about in class.

3) Create a vocabulary using the X_train dataset and transform both your X_train and X_test data using that vocabulary.

4) Use your choice of binary classification algorithm to train and evaluate your model's accuracy. Report both train and test accuracies.

 - Stretch goal: Use an error metric other than accuracy and implement/evaluate multiple classifiers.
 - Stretch goal: Track your results in a DataFrmae and produce a visualization of the results



In [12]:
# Split
X = twitter['CleanedText']
y = twitter['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(79991,)
(19998,)
(79991,)
(19998,)


In [0]:
# CountVectorizer
count_vectorizer = CountVectorizer(stop_words = stop)
# Learn vocabulary from training texts and vectorize training & test texts.
X_train_word_counts = count_vectorizer.fit_transform(X_train)
X_test_word_counts = count_vectorizer.transform(X_test)

In [14]:
# LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_word_counts, y_train)

print("Train ROC AUC:", roc_auc_score(lr.predict(X_train_word_counts), y_train))
print("Test ROC AUC:", roc_auc_score(lr.predict(X_test_word_counts), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train ROC AUC: 0.908334221220438
Test ROC AUC: 0.756417932172525


In [18]:
# MultinomialNB
mnb = MultinomialNB().fit(X_train_word_counts, y_train)
mnb.fit(X_train_word_counts, y_train);

print("Train ROC AUC:", roc_auc_score(mnb.predict(X_train_word_counts), y_train))
print("Test ROC AUC:", roc_auc_score(mnb.predict(X_test_word_counts), y_test))

Train ROC AUC: 0.894478826483809
Test ROC AUC: 0.7502737151203479


In [0]:
# TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words = stop)
# Learn vocabulary from training texts and vectorize training & test texts.
X_train_tfidf_word_counts = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf_word_counts = tfidf_vectorizer.transform(X_test)

In [20]:
# LogisticRegression
LR = LogisticRegression()
LR.fit(X_train_tfidf_word_counts, y_train)

print("Train ROC AUC:", roc_auc_score(LR.predict(X_train_tfidf_word_counts), y_train))
print("Test ROC AUC:", roc_auc_score(LR.predict(X_test_tfidf_word_counts), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train ROC AUC: 0.8406775691692335
Test ROC AUC: 0.7605208478193384


In [22]:
# MultinomialNB
MNB = MultinomialNB().fit(X_train_tfidf_word_counts, y_train)
MNB.fit(X_train_tfidf_word_counts, y_train);

print("Train ROC AUC:", roc_auc_score(MNB.predict(X_train_tfidf_word_counts), y_train))
print("Test ROC AUC:", roc_auc_score(MNB.predict(X_test_tfidf_word_counts), y_test))

Train ROC AUC: 0.8969721824016295
Test ROC AUC: 0.7493424882623017


# Part 4 -  Word2Vec

1) Fit a Word2Vec model on your cleaned/tokenized twitter dataset. 

2) Display the 10 words that are most similar to the word "twitter"

In [23]:
model = Word2Vec(twitter['Tokens'])
model.most_similar('twitter')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('list', 0.8170230388641357),
 ('facebook', 0.7924103140830994),
 ('link', 0.7875146865844727),
 ('account', 0.7847198843955994),
 ('dm', 0.784254252910614),
 ('forces', 0.7831788063049316),
 ('page', 0.7787084579467773),
 ('following', 0.7736862897872925),
 ('chrisbosh', 0.7690141201019287),
 ('world', 0.7687739133834839)]